In [1]:
import urllib.request
import json
import openai
import json
from openai import OpenAI
from openai.types import Completion, CompletionChoice, CompletionUsage

In [2]:
import spotipy
import webbrowser
import geocoder
from nltk.corpus import stopwords
import spotipy.util as util
import random
import requests
import datetime


In [3]:
#Get ISS position API
request = urllib.request.Request("http://api.open-notify.org/iss-now.json")
response = urllib.request.urlopen(request)

In [4]:
iss_obj = json.loads(response.read())

In [5]:
# The International Space Station moves at 28,000 km/h 
line = 'ISS'
timestamp = iss_obj['timestamp']
if line == 'ISS':
    latitude=iss_obj['iss_position']['latitude']
    longitude=iss_obj['iss_position']['longitude']
time=datetime.datetime.fromtimestamp(iss_obj['timestamp'])
print(latitude,longitude,time)

13.4134 -76.2124 2023-11-30 02:53:03


In [6]:
with open("keys/city_key.txt", "r") as file:
    city_key = file.read()

In [7]:
#Get location information
requestCity = urllib.request.Request("https://api.opencagedata.com/geocode/v1/json?q="+latitude+","+longitude+"&key="+city_key)
responseCity = urllib.request.urlopen(requestCity)


In [8]:
city_obj = json.loads(responseCity.read())

In [9]:
print(city_obj)

{'documentation': 'https://opencagedata.com/api', 'licenses': [{'name': 'see attribution guide', 'url': 'https://opencagedata.com/credits'}], 'rate': {'limit': 2500, 'remaining': 2497, 'reset': 1701388800}, 'results': [{'annotations': {'DMS': {'lat': "13° 24' 48.24000'' N", 'lng': "76° 12' 44.64000'' W"}, 'MGRS': '18PUV6873783175', 'Maidenhead': 'FK13vj49mf', 'Mercator': {'x': -8483925.56, 'y': 1497096.042}, 'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=17/13.41340/-76.21240&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=13.41340&mlon=-76.21240#map=17/13.41340/-76.21240'}, 'geohash': 'd63u39zfdqgxwt8n3bup', 'qibla': 61.72, 'roadinfo': {'drive_on': 'right', 'speed_in': 'km/h'}, 'sun': {'rise': {'apparent': 1701342720, 'astronomical': 1701338220, 'civil': 1701341400, 'nautical': 1701339780}, 'set': {'apparent': 1701383700, 'astronomical': 1701388200, 'civil': 1701385020, 'nautical': 1701386640}}, 'timezone': {'name': 'GMT+5', 'now_in_dst': 0, 'offset_sec': -1800

In [10]:
#Get the location name
category=city_obj["results"][0]["components"]["_category"]

type=city_obj["results"][0]["components"]["_type"]
if type=='building' or type=='road':
    location=city_obj["results"][0]["components"]["county"]
else:
    location=city_obj["results"][0]["components"][type]
    

print(location)

Caribbean Sea


In [11]:
#To check the day time or the night time by the time of sunrise and sunset
loc_sun=city_obj["results"][0]['annotations']['sun']
sunrise=loc_sun['rise']['apparent']
sunset=loc_sun['set']['apparent']
localtime=timestamp+city_obj["results"][0]['annotations']['timezone']['offset_sec']

if localtime>sunrise and localtime<sunset: 
    city_status='day'
else:
    city_status='night'
    
print(city_status)

night


In [12]:

with open("keys/weather_key.txt", "r") as file:
    weather_key = file.read()
    
with open("keys/openai_key.txt", "r") as file:
    openai_key = file.read()

In [13]:
#Get local weather information
weather_url = f'https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={weather_key}'
request_weather = urllib.request.Request(weather_url)
response_weather = urllib.request.urlopen(request_weather)

In [14]:
weather_obj = json.loads(response_weather.read())
weather = weather_obj['weather'][0]['main']
print (weather)

Clear


In [26]:
#Get key words by OpenAI API
number='20'

client = OpenAI(
   
    api_key= openai_key,
)
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Generate JSON list of {number} keywords for playlist of {location} in {city_status} time in {weather} weather, with no format, no explanation, no enter.", 

        }
    ],
    model="gpt-3.5-turbo",
)
print(response)


ChatCompletion(id='chatcmpl-8QRWY21CTjLoUYufmPoqVI9XkmLQR', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='["Caribbean", "sea", "night", "clear", "weather", "moonlit", "waves", "stars", "ocean", "serene", "tranquil", "peaceful", "relaxing", "sparkling", "magical", "tropical", "calm", "island", "romantic", "paradise"]', role='assistant', function_call=None, tool_calls=None))], created=1701313022, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=76, prompt_tokens=36, total_tokens=112))


In [16]:
#Format the response
raw_string = str(response)

if raw_string.find("keywords")>0:
    content_start = raw_string.find('"keywords":' ) + len('"keywords":')
    content_end = raw_string.find("', role=", content_start)-1
else:
    content_start = raw_string.find("content='") + len("content='")
    content_end = raw_string.find("', role=", content_start)
json_content = raw_string[content_start:content_end]


print(json_content)
keywords_list = json.loads(json_content)

print(keywords_list)


["Caribbean", "Nighttime", "Sea", "Travel", "Relaxation", "Beaches", "Islands", "Ocean", "Moonlight", "Sandy shores", "Palm trees", "Starlit sky", "Tropical", "Euphoria", "Calmness", "Serene", "Paradise", "Hammocks", "Crashing waves", "Escape"]
['Caribbean', 'Nighttime', 'Sea', 'Travel', 'Relaxation', 'Beaches', 'Islands', 'Ocean', 'Moonlight', 'Sandy shores', 'Palm trees', 'Starlit sky', 'Tropical', 'Euphoria', 'Calmness', 'Serene', 'Paradise', 'Hammocks', 'Crashing waves', 'Escape']


In [17]:
with open("keys/spotify_keys.json", "r") as file:
    api_tokens = json.load(file)

In [18]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]
userid = api_tokens["userid"]

In [19]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [20]:
sp = spotipy.Spotify(auth=token)

In [21]:
tracks = []

for word in keywords_list:
    searchResults = sp.search(q=word,type='track',limit=1)
    if len(searchResults['tracks']['items'])>0:
        tracks.append(searchResults['tracks']['items'][0]['uri'])

In [24]:
image_id='https://d13k13wj6adfdf.cloudfront.net/urban_areas/san-francisco-bay-area_web-f17b1f60e6.jpg'
my_playlist = sp.user_playlist_create(user=userid, name=f"Go to {location} with {line}", public=True,
                                      description=f"Welcome on board ISS line, we are now approching {location}, hope you have a good journey!  Arrrival Time: {time}  Weather: {weather}")

results = sp.user_playlist_add_tracks(username, my_playlist['id'], tracks)
print(results)
print(my_playlist)

{'snapshot_id': 'Myw3YmEzNzAwNWQ4MWU0NzY4NDZkMzYwZmVkNTAzMmUyMTM3YzU2ZTZh'}
{'collaborative': False, 'description': 'Welcome on board ISS line, we are now approching Caribbean Sea, hope you have a good journey!  Arrrival Time: 2023-11-30 02:53:03  Weather: Clear', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/14wmJ1G7VLmPBzc7yEiAwU'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/14wmJ1G7VLmPBzc7yEiAwU', 'id': '14wmJ1G7VLmPBzc7yEiAwU', 'images': [], 'name': 'Go to Caribbean Sea with ISS', 'owner': {'display_name': 'WangxinLiu', 'external_urls': {'spotify': 'https://open.spotify.com/user/31mvdqp55nskpk66v4nzpyea42ci'}, 'href': 'https://api.spotify.com/v1/users/31mvdqp55nskpk66v4nzpyea42ci', 'id': '31mvdqp55nskpk66v4nzpyea42ci', 'type': 'user', 'uri': 'spotify:user:31mvdqp55nskpk66v4nzpyea42ci'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MSxkNTBlZjc2YzU2NmQwMTJlZjY0MTJiOWMyMWI0OWYxYTU2MTMyMDNm', 'tracks': {'href': 'h

In [23]:
webbrowser.open(my_playlist['external_urls']['spotify'])

True